In [1]:
import os
import sys

# Note that the path to spark home could be different for you. This one is for HortonWorks Data Platform
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"

# The version for py4j could be different for you
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell'
# com.databricks:spark-avro_2.11:3.2.0
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,com.databricks:spark-avro_2.11:3.2.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path ~/mysql-connector-java-5.1.36-bin.jar --jars ~/mysql-connector-java-5.1.36-bin.jar --packages com.databricks:spark-xml_2.11:0.6.0,com.databricks:spark-avro_2.11:3.2.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()
sc = spark.sparkContext
# specify .master("yarn")

In [3]:
from pyspark.sql import Row
import re

regex = '^([^ ]+) \- \- \[(.+)\] "(.+) (.+) (.+)" ([0-9]+) ([0-9]+)'

def toarry(line):
    m = re.match(regex, line)
    if m:
        g = m.groups()
        if len(g) > 0:
            return [Row(host=g[0], datetime=g[1], type=g[2], url=g[3], retcode=g[5], timetaken=g[6])]
    return []


lines = sc.textFile("/data/spark/project/access/")
rows = lines.flatMap(toarry)


In [4]:
rows.count()

1612715

In [5]:
df = rows.toDF()
df.show()

+--------------------+--------------+-------+---------+-------+--------------------+
|            datetime|          host|retcode|timetaken|   type|                 url|
+--------------------+--------------+-------+---------+-------+--------------------+
|27/Sep/2015:06:48...|           ::1|    200|      125|OPTIONS|                   *|
|27/Sep/2015:06:48...|           ::1|    200|      125|OPTIONS|                   *|
|27/Sep/2015:06:48...|           ::1|    200|      125|OPTIONS|                   *|
|27/Sep/2015:06:48...|  66.249.65.26|    200|     7507|    GET|/node/538/done?si...|
|27/Sep/2015:06:49...|107.170.18.142|    304|      125|    GET|/sites/default/fi...|
|27/Sep/2015:06:49...|107.170.18.142|    304|      125|    GET|/sites/default/fi...|
|27/Sep/2015:06:49...|107.170.18.142|    304|      125|    GET|/sites/default/fi...|
|27/Sep/2015:06:49...|107.170.18.142|    304|      125|    GET|/sites/default/fi...|
|27/Sep/2015:06:49...|107.170.18.142|    304|      125|    GET|/s